# Fulmine LABS Eyball
## Overview
This Python code implements a class wrapper around an Anomaly Detection model which can be used to visually check if an image is anomalous or not. The supported architecture for this model is 'Siamese Network'.
In order to perform reduce false negatives the code compares the image against a jury of randomly selected known good images of configurable size 'jury_size'. These images were studies obtained from the Cancer Image Database, stored in Orthanc and then modified using the Fulmine mini-PACS code.
If the number of jurors who vote that the image is simlar to the chosen known good image is below a configurable 'threshold' then the code returns a verdict of 'Anomalous', otherwise it returns a verdict of 'Normal'.
If an image path is not specified but screen coordinates are, these will be used instead, enabling direct integration with automated visual checking scripts.

One goal is to use this class as part of automating visual checking of a medical image (PACS) production pipeline, although it could theoretically visually check any type of image on which the model has been trained.

It also has the capability of describing the images, using GPT-4 Turbo Vision, if an OpenAI key is supplied in the 'Eyball-OpenAI_key.txt' file.

## Initialize the Eyball class

predictor = ModelPredictor(siamese_model_path, known_good_images_folder, Eyball_key, threshold, jury_size)

## Example calls

role = "You are a radiology PACS test engineer, analyzing PACS or test process related image anomalies"

image_description_directive = "If the image is obviously not a medical image, state *** ANOMALOUS ***. If it is a typical medical image as acquired by an imaging modality with no additions or enhancements, state *** NORMAL ***. Otherwise, if it is a medical image but it also clearly has textual overlays or annotations or digital or image processing artifacts that could have been added by the PACS image viewer technology, describe those features and append *** ANOMALOUS ***."

verdict = predictor.predict_siamese(test_image_path)

actual_description = predictor.describe_image(test_image_path, None, role, image_description_directive)

## Author
Duncan Henderson
Fulmine Labs LLC


In [1]:
import numpy as np
import os
import io
import cv2
from PIL import Image, ImageGrab
import logging
import random
import base64
import requests
from openai import OpenAI

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report



In [2]:
known_good_images_folder = r"d:\training_images\test\valid" # Generated by running the Fulmine mini-PACS archive code on Orthanc studies
siamese_model_path = r'models\lung_ct_siamese_network_weights_043024.h5' # Generated by running the Fulmine mini-PACS model generation code

api_key_file='Eyball-OpenAI_key.txt'

jury_size=12
threshold = 0.5

# LLM prompts
role = "You are a radiology PACS test engineer, analyzing PACS or test process related image anomalies"
image_description_directive = "If the image is obviously not a medical image, state *** ANOMALOUS ***. If it is a typical medical image as acquired by an imaging modality with no additions or enhancements, state *** NORMAL ***. Otherwise, if it is a medical image but it also clearly has textual overlays or annotations or digital or image processing artifacts that could have been added by the PACS image viewer technology, describe those features and append *** ANOMALOUS ***."


In [3]:
import os
import numpy as np
import cv2
from PIL import Image, ImageGrab
import base64
import io
import requests
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
import tensorflow as tf
from openai import OpenAI

class ModelPredictor:
    def __init__(self, siamese_model_path, known_good_images_folder, api_key_file='Eyball-OpenAI_key.txt', threshold=0.5, jury_size=12):
        self.siamese_model_path = siamese_model_path
        self.known_good_images_folder = known_good_images_folder
        self.api_key = self.load_api_key(api_key_file)
        self.client = OpenAI(api_key=self.api_key)
        self.siamese_model = self.load_siamese_model()
        self.threshold = threshold
        self.jury_size = jury_size
        self.known_good_images = self.preload_known_good_images()
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }

    def load_api_key(self, filename):
        try:
            with open(filename, 'r') as file:
                return file.read().strip()
        except FileNotFoundError:
            raise Exception(f"API key file not found: {filename}")

    def preload_known_good_images(self):
        print("Preloading known good images...")
        image_paths = []
        for root, dirs, files in os.walk(self.known_good_images_folder):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    full_path = os.path.join(root, file)
                    image_paths.append(full_path)
        return image_paths

    def load_siamese_model(self):
        input_shape = (152, 152, 1)
        input_a = Input(shape=input_shape)
        input_b = Input(shape=input_shape)
        base_network = self.initialize_base_network(input_shape)
        processed_a = base_network(input_a)
        processed_b = base_network(input_b)
        distance = Lambda(lambda tensors: tf.sqrt(tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1)))([processed_a, processed_b])
        model = Model(inputs=[input_a, input_b], outputs=distance)
        model.load_weights(self.siamese_model_path)
        print("Siamese model loaded successfully.")
        return model

    def initialize_base_network(self, input_shape):
        input = Input(shape=input_shape)
        x = Conv2D(64, (3, 3), activation='relu')(input)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(128, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        return Model(inputs=input, outputs=x)

    
    def preprocess_image(self, image=None, image_path=None):
        if image_path:
            # Load image from file path
            try:
                image = Image.open(image_path)
            except FileNotFoundError as e:
                print(f"Failed to open image at {image_path}: {e}")
                return None
            except Exception as e:
                print(f"Error processing image at {image_path}: {e}")
                return None
        if isinstance(image, np.ndarray):
            # Convert to PIL Image for consistent processing
            image = Image.fromarray(image.astype('uint8'))
    
        if image is None:
            print("No image provided for preprocessing")
            return None
    
        # Convert to grayscale and resize
        image = image.convert('L')  # Convert to grayscale
        target_size = (152, 152)
        image = image.resize(target_size)
        image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
        if image.ndim == 2:  # Ensure image has 3 dimensions if it's still 2D
            image = np.expand_dims(image, -1)
    
        return image

    def capture_screen(self, coordinates):
        screenshot = ImageGrab.grab(bbox=coordinates)
        return np.array(screenshot, dtype=np.uint8)  # Ensure dtype is uint8

    def predict_siamese(self, image_path=None, coordinates=None):
        if coordinates:
            print("Capturing screen...")
            captured_image = self.capture_screen(coordinates)
            if captured_image is None:
                print("Failed to capture screen")
                return None
            image = self.preprocess_image(image=captured_image)
        elif image_path:
            image = self.preprocess_image(image_path=image_path)
            if image is None:
                print("Failed to preprocess image from path")
                return None
        else:
            raise ValueError("Either image_path or coordinates must be provided.")

    def predict_siamese(self, image_path=None, coordinates=None):
        if coordinates:
            print("Capturing screen...")
            captured_image = self.capture_screen(coordinates)
            if captured_image is None:
                print("Failed to capture screen")
                return None
            image = self.preprocess_image(image=captured_image)
        elif image_path:
            image = self.preprocess_image(image_path=image_path)
        else:
            raise ValueError("Either image_path or coordinates must be provided.")
    
        if image is None:
            print("No image to process")
            return None
    
        image = np.expand_dims(image, axis=0)  # Adjust as necessary for the model input
    
        print("Image loaded and processed, predicting...")

        votes = []
        for known_good_image_path in random.sample(self.known_good_images, min(self.jury_size, len(self.known_good_images))):
            known_good_image = self.preprocess_image(image_path=known_good_image_path)
            if known_good_image is None:
                continue  # Skip if image can't be processed
            known_good_image = np.expand_dims(known_good_image, axis=0)

            # Prepare the pair
            image_pair = [image, known_good_image]

            # Make prediction
            prediction_distance = self.siamese_model.predict(image_pair)
            is_similar = prediction_distance < self.threshold  # Threshold to determine similarity
            print(f"Comparing {image_path if image_path else 'screen capture'} with {known_good_image_path}: Distance = {prediction_distance}, Similar = {is_similar}")
            votes.append(is_similar)

        # Calculate the majority vote
        num_similar = sum(votes)
        majority_similar = num_similar > len(votes) / 2
        print(f"Total votes for 'Similar': {num_similar}/{len(votes)}. Final verdict: {'Normal' if majority_similar else 'Anomalous'}")

        return 'Normal' if majority_similar else 'Anomalous'

    def describe_image(self, image_path=None, coordinates=None, role_description="User", image_description_directive="Describe the image"):
        try:
            if coordinates:
                print("Capturing screen for description...")
                image = self.capture_screen(coordinates)
            elif image_path:
                print("Loading image from path for description...")
                image = self.preprocess_image(image_path=image_path)
            else:
                raise ValueError("Either image_path or coordinates must be provided.")
    
            if image is None:
                raise ValueError("Failed to load or process image.")
    
            print("Encoding image for API request...")
            base64_image = self.encode_image(image)
            description = self.send_image_to_api(base64_image, role_description, image_description_directive)
            print("Description received.")
            return description
        except Exception as e:
            print(f"Error in describe_image: {str(e)}")
            return None

    def encode_image(self, image):
        """Converts a numpy array image to JPEG base64."""
        try:
            if image.ndim == 3 and image.shape[2] == 1:  # Check if it's single-channel
                image = image.squeeze(-1)  # Remove the last dimension if it's single-channel
            if isinstance(image, np.ndarray):
                # Ensure the data type is uint8
                image = (image * 255).clip(0, 255).astype(np.uint8)
                # Convert numpy array to PIL Image
                if image.ndim == 2:  # Grayscale
                    image = Image.fromarray(image, 'L')
                else:
                    image = Image.fromarray(image, 'RGB')
            buffer = io.BytesIO()
            image.save(buffer, format="JPEG")
            encoded_string = base64.b64encode(buffer.getvalue()).decode('utf-8')
            return encoded_string
        except Exception as e:
            raise ValueError(f"Error encoding image: {str(e)}")

    def send_image_to_api(self, base64_image, role_description, image_description_directive):
        print("Sending image to API...")
        payload = {
            "model": "gpt-4-turbo",
            "messages": [
                {
                    "role": "system",
                    "content": role_description
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": image_description_directive
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=self.headers, json=payload)
        if response.status_code != 200:
            print("Error from API:", response.status_code, response.text)
            return None

        try:
            description = response.json()['choices'][0]['message']['content']
            return description
        except KeyError as e:
            print("Failed to parse API response:", response.json())
            raise e



In [4]:
predictor = ModelPredictor(siamese_model_path, known_good_images_folder, api_key_file, threshold, jury_size)



Siamese model loaded successfully.
Preloading known good images...


In [5]:
# Capture and evaluate an area of the screen ...

left = 10
right = 500
top = 10
bottom = 200

siamese_result = predictor.predict_siamese(coordinates=(left, top, right, bottom))
print ("Siamese result", siamese_result)

predictor.describe_image(coordinates=(left, top, right, bottom), role_description=role, image_description_directive=image_description_directive)

Capturing screen...
Image loaded and processed, predicting...
1/1 [==============================] - 0s 90ms/step
Comparing screen capture with d:\training_images\test\valid\dummy_class\843f76f3-0cb6-4501-ab7b-a67455b347d9.png: Distance = [1.030087], Similar = [False]
1/1 [==============================] - 0s 23ms/step
Comparing screen capture with d:\training_images\test\valid\randomized_wl\cropped\cdba8ea1-4d2d-40e9-9102-223c07963157_1.png: Distance = [1.030087], Similar = [False]
1/1 [==============================] - 0s 23ms/step
Comparing screen capture with d:\training_images\test\valid\randomized_wl\cropped\9b6bb52c-bdee-4324-acc6-5c434edbdc0a_0.png: Distance = [1.030087], Similar = [False]
1/1 [==============================] - 0s 28ms/step
Comparing screen capture with d:\training_images\test\valid\randomized_wl\cropped\de32d902-a3d0-4605-bdc3-a8020a982cd6_0.png: Distance = [1.030087], Similar = [False]
1/1 [==============================] - 0s 22ms/step
Comparing screen captu

'*** ANOMALOUS ***\n\nThis image is not a medical image. It appears to be a screenshot of a computer interface, possibly related to image generation software, showing various tabs and a text input box with narrative content. There are no features typical of medical imaging like anatomical structures or radiological scans.'

In [6]:
# Or (from here on) pass in a captured and saved file

test_image_path = r'C:\temp\engineer_typing3.png'
print("Model predicts", predictor.predict_siamese(image_path=test_image_path))

actual_description = predictor.describe_image(image_path=test_image_path, role_description=role, image_description_directive=image_description_directive)
print ("LLM description", actual_description)


Image loaded and processed, predicting...
1/1 [==============================] - 0s 43ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\zoomed\randomized_wl\1e14e1d7-2edc-49c6-a27f-a0f016a043bc_1.png: Distance = [1.0572661], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\zoomed\randomized_wl\aae1be57-2f16-4c6c-afb9-bce376dc3044_0.png: Distance = [1.0572661], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\randomized_wl\cropped\3c13562b-adc3-43a4-8feb-45168aa6a9fa_1.png: Distance = [0.7900274], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\zoomed\randomized_wl\1a4f34db-934b-4559-909e-b5d577825cd4_0.png: Distance = [1.0572661], Similar = [False]
1/1 [====================

In [7]:
test_image_path = r'Custom_invalid\cat.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\randomized_wl\cropped\c59d9a44-5fc2-4034-95f6-b37a25e964b4_1.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 44ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\randomized_wl\cropped\48aab34c-a2c9-4f92-940b-76ddccd9a0f1_0.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 44ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\zoomed\randomized_wl\1cad5371-2ae3-466a-9a8a-4b5558569bdd_1.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\randomized_wl\cropped\3e2c8f1b-5b7c-4a4b-9005-0d1864e2646a_1.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 45ms/step

In [8]:
test_image_path = r'custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\training_images\test\valid\dummy_class\234985a4-b400-4dd8-85a4-9885251dee0c.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 42ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\training_images\test\valid\randomized_wl\cropped\10b7de92-365e-424d-adc0-7492f90aab91_0.png: Distance = [0.37830558], Similar = [ True]
1/1 [==============================] - 0s 44ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\training_images\test\valid\zoomed\randomized_wl\41026ec9-e191-4568-aeb6-f51c44bf0915_1.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 42ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\training_images\tes

In [9]:
test_image_path = r'custom_test_valid\istockphoto-493741910-612x612.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\randomized_wl\cropped\cfaa6e2c-bb04-4d99-a74b-070ce5673469_1.png: Distance = [0.11931601], Similar = [ True]
1/1 [==============================] - 0s 40ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\randomized_wl\cropped\c041e1ee-a4ea-43be-aeb9-2826c8db9413_1.png: Distance = [0.14590386], Similar = [ True]
1/1 [==============================] - 0s 43ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\zoomed\randomized_wl\e2ef633b-06a5-4bce-b2ee-b4e97a666fe4_0.png: Distance = [0.11514193], Similar = [ True]
1/1 [==============================] - 0s 42ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\dummy_class\fe1f1249-bb0d-4c6a-a08b-

In [10]:
test_image_path = r'custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 43ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with d:\training_images\test\valid\zoomed\randomized_wl\f3181ae2-bbec-4644-a38f-a33b00489be5_1.png: Distance = [0.71923923], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with d:\training_images\test\valid\randomized_wl\cropped\4b665f09-3ef8-4d90-8368-1ed44796fbe9_1.png: Distance = [0.71923923], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with d:\training_images\test\valid\zoomed\randomized_wl\d369fc1f-368a-4fc4-9885-a96848146cf4_1.png: Distance = [0.6875133], Similar = [False]
1/1 [==============================] - 0s 45ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.j

In [11]:
test_image_path = r'custom_invalid\istockphoto-with_arrow.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 42ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\randomized_wl\cropped\b4d0d11a-ce1a-454d-8aa4-d13f3081e44a_0.png: Distance = [0.55774856], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\zoomed\randomized_wl\9afdfc09-bb00-4153-90de-1bd2556f0b98_1.png: Distance = [0.8484544], Similar = [False]
1/1 [==============================] - 0s 44ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\zoomed\randomized_wl\e6cbcf4e-a264-4a98-a3a5-aa3103615a03_0.png: Distance = [0.8484544], Similar = [False]
1/1 [==============================] - 0s 40ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\dummy_class\733dc5ac-9793-4a7f-afef-bd4abd186d8d.png: Distance = [0.8484544], S

In [12]:
test_image_path = r'custom_invalid\Lung_abscess_-_CT_with_overlay.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 40ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\randomized_wl\cropped\7e11f3b3-2e37-4fdf-b9f3-b4f4c8cb60e5_1.png: Distance = [1.1311321], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\zoomed\randomized_wl\6a47da6e-b3ea-4c65-88fc-f8ee062d9375_1.png: Distance = [1.1311321], Similar = [False]
1/1 [==============================] - 0s 44ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\randomized_wl\cropped\d27c0207-792d-4a1c-9892-47f29f811e75_0.png: Distance = [0.9937784], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\zoomed\randomized_wl\d8b71f80-48f3-449a-95bb-95

In [13]:
test_image_path = r'Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\valid\zoomed\randomized_wl\7c04ad64-9d77-4d95-9409-552780caa4a1_1.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\valid\dummy_class\ad7663fd-617a-4cfd-a8c9-5dcbdb370dce.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\valid\randomized_wl\cropped\89091e5a-2f7c-4f07-b0b9-e8998af20388_1.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\v

In [14]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with d:\training_images\test\valid\randomized_wl\cropped\9904c6d1-21eb-4121-a918-7cf329a1f4b9_1.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 44ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with d:\training_images\test\valid\zoomed\randomized_wl\c6d414ce-04a2-4502-9dc1-fcd4faee5fe8_1.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with d:\training_images\test\valid\zoomed\randomized_wl\3e7e08f0-78b9-483c-9fd8-ae69c7c851ff_0.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 40ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modi

In [15]:
test_image_path = r'Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))

Image loaded and processed, predicting...
1/1 [==============================] - 0s 42ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with d:\training_images\test\valid\dummy_class\cc7c972f-8f9e-49bb-9750-506d30a9ccac.png: Distance = [0.77528584], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with d:\training_images\test\valid\randomized_wl\cropped\249b0656-7f63-45a0-b779-75d3111601dd_1.png: Distance = [0.77528584], Similar = [False]
1/1 [==============================] - 0s 44ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with d:\training_images\test\valid\zoomed\randomized_wl\ef335ad6-ba5c-40d7-bf5f-b17cb4b54be7_0.png: Distance = [0.6745002], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with d:\

In [16]:
test_image_path = r'Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 40ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\randomized_wl\cropped\5ca0a77b-3ea9-45cd-881b-6988680eb879_1.png: Distance = [0.45649433], Similar = [ True]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\dummy_class\f73b6dd0-e75a-4c9e-b5f7-d3302ba08fd0.png: Distance = [0.45649433], Similar = [ True]
1/1 [==============================] - 0s 45ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\zoomed\randomized_wl\acc7c15e-e918-4724-80f9-7b2042a39a17_0.png: Distance = [0.45649433], Similar = [ True]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\zoomed\r

In [17]:
def evaluate_methods_simplified(base_folder, sample_size=40, jury_size=12, role="User", image_description_directive="Describe the image"):
    valid_folder = os.path.join(base_folder, 'valid')
    invalid_folder = os.path.join(base_folder, 'invalid')

    # Ensure directories exist
    if not os.path.exists(valid_folder) or not os.path.exists(invalid_folder):
        raise ValueError("One or more image directories do not exist.")

    valid_images = random.sample(os.listdir(valid_folder), min(sample_size, len(os.listdir(valid_folder))))
    invalid_images = random.sample(os.listdir(invalid_folder), min(sample_size, len(os.listdir(invalid_folder))))
    
    predictions_siamese = []
    predictions_gpt = []
    actuals = [1] * len(valid_images) + [0] * len(invalid_images)

    for filename in valid_images + invalid_images:
        folder = valid_folder if filename in valid_images else invalid_folder
        image_path = os.path.join(folder, filename)

        siamese_result = predictor.predict_siamese(image_path)
        print ("Siamese prediction", siamese_result)
        predictions_siamese.append(1 if siamese_result == 'Normal' else 0)

        description_result = predictor.describe_image(image_path, None, role, image_description_directive)
        print ("LLM prediction", description_result)
        predictions_gpt.append(1 if 'NORMAL' in description_result.upper() else 0)

    accuracy_s = accuracy_score(actuals, predictions_siamese)
    precision_s = precision_score(actuals, predictions_siamese)
    recall_s = recall_score(actuals, predictions_siamese)
    f1_s = f1_score(actuals, predictions_siamese)
    
    accuracy_g = accuracy_score(actuals, predictions_gpt)
    precision_g = precision_score(actuals, predictions_gpt)
    recall_g = recall_score(actuals, predictions_gpt)
    f1_g = f1_score(actuals, predictions_gpt)

    print('Evaluation Results - Siamese Model:', {'accuracy': accuracy_s, 'precision': precision_s, 'recall': recall_s, 'f1': f1_s})
    print('Evaluation Results - GPT Model:', {'accuracy': accuracy_g, 'precision': precision_g, 'recall': recall_g, 'f1': f1_g})

    return {
        "siamese": {"accuracy": accuracy_s, "precision": precision_s, "recall": recall_s, "f1": f1_s},
        "gpt": {"accuracy": accuracy_g, "precision": precision_g, "recall": recall_g, "f1": f1_g}
    }



In [ ]:
# Example call
base_folder = r'model_comparison_test'
sample_size = 40
try:
    results = evaluate_methods_simplified(base_folder, sample_size, jury_size, role, image_description_directive)
    print("Evaluation Results:", results)
except Exception as e:
    print("Error during evaluation:", str(e))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 43ms/step
Comparing model_comparison_test\valid\f9554b0d-77bc-4b51-b044-7fc8918a614b.png with d:\training_images\test\valid\zoomed\randomized_wl\733dc5ac-9793-4a7f-afef-bd4abd186d8d_0.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 41ms/step
Comparing model_comparison_test\valid\f9554b0d-77bc-4b51-b044-7fc8918a614b.png with d:\training_images\test\valid\zoomed\randomized_wl\965fc321-1dab-4285-9965-e1b6f4db5a24_0.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 41ms/step
Comparing model_comparison_test\valid\f9554b0d-77bc-4b51-b044-7fc8918a614b.png with d:\training_images\test\valid\dummy_class\8e8090ac-e850-493e-83a9-88700c6aa03d.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 39ms/step
Comparing model_comparison_test\valid\f9554b0d-77bc-4b51-b044-7fc8918a614b.png with d:\training_images\test\valid\rando